In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [2]:
import pandas as pd
import numpy as np

In [3]:
import warnings

warnings.filterwarnings("ignore", message="DataFrame is highly fragmented.", category=pd.errors.PerformanceWarning)

In [1]:
import pandas as pd
import numpy as np
import requests
from urllib.parse import urlencode
from dotenv import load_dotenv
import os
import math
from itertools import chain

pd.options.display.max_columns=999

## 1. Authorization Flow

In [9]:
from src.data_download.api_utils import authorize_user

In [10]:
authorize_user()

Go to the following URL and authorize access: https://accounts.spotify.com/authorize?client_id=70dabab8a4f44e8c9014cc7b8f86d086&response_type=code&redirect_uri=https%3A%2F%2Foauth.pstmn.io%2Fv1%2Fbrowser-callback&scope=user-library-read


'https://accounts.spotify.com/authorize?client_id=70dabab8a4f44e8c9014cc7b8f86d086&response_type=code&redirect_uri=https%3A%2F%2Foauth.pstmn.io%2Fv1%2Fbrowser-callback&scope=user-library-read'

In [11]:
auth_code="AQBiYAo8rUOKaFvQvdnN9i0EV2Yu2fJ9_lkoMGQW-WLjdJuHTSbwSmwvAjmi1qZtvCIY8bmFtPbT6kEk5AMm8Kkzlk0KigcIFXx7-BgnAzHmrPfPQj6wv5zPr80-0WATT35QDkY9CMiRY-V5olyi2voJn2aEBc_JCClK0TEfeZlWcYL1YdpvwwlfQaAArC-KPv-TLJ4WGYsFf4HgHKFw8Ns"

In [12]:
from src.data_download.api_utils import SpotifyAPI

In [13]:
spotify_api = SpotifyAPI(auth_code=auth_code)

In [14]:
spotify_api.get_access_token()

## 2. Making API Requests

In [15]:
track_id = "1BSq0hYijZbubyR6waNyQn"

endpoint = f"audio-features/{track_id}"
response = spotify_api.execute_api_request(endpoint)

if response.status_code != 200:
    raise HTTPError(response.status_code)

In [16]:
response.status_code

200

In [17]:
response.json().keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [18]:
response.json()['danceability']

0.784

In [19]:
response.json()

{'danceability': 0.784,
 'energy': 0.402,
 'key': 9,
 'loudness': -9.719,
 'mode': 0,
 'speechiness': 0.324,
 'acousticness': 0.0287,
 'instrumentalness': 0.0071,
 'liveness': 0.0963,
 'valence': 0.402,
 'tempo': 166.678,
 'type': 'audio_features',
 'id': '1BSq0hYijZbubyR6waNyQn',
 'uri': 'spotify:track:1BSq0hYijZbubyR6waNyQn',
 'track_href': 'https://api.spotify.com/v1/tracks/1BSq0hYijZbubyR6waNyQn',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1BSq0hYijZbubyR6waNyQn',
 'duration_ms': 335800,
 'time_signature': 4}

## 3. Get Library Tracks

In [24]:
tracks = spotify_api.get_user_library_tracks()

In [26]:
type(tracks)

list

In [27]:
tracks[0].keys()

dict_keys(['added_at', 'track'])

In [31]:
tracks[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [33]:
def get_entire_user_library():
    """
    Retrieves the entire library of tracks saved 
    in the authenticated user's library.

    This function iterates through all pages of the user's 
    library tracks, retrieving tracks in batches of 50 
    until all tracks have been retrieved.

    Returns:
    list: A list of track objects representing the user's entire library.
    """
    tracks = []
    offset=0
    length = 1
    while length > 0:
        new_tracks = spotify_apiget_user_library_tracks(offset=offset)
        length = len(new_tracks) # if no tracks returned, break the while loop
        tracks += new_tracks
        offset += 50
    return tracks

In [34]:
library = get_entire_user_library()
len(library)

5742

In [36]:
library[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [37]:
def process_library_tracks(library):
    """
    Processes a list of track information from the user's library 
    and converts it into a pandas DataFrame.

    This function extracts relevant track and album details 
    from the provided library data and organizes
    them into a structured DataFrame for further analysis or manipulation.

    Parameters:
    library (list): A list of dictionaries where each dictionary contains 
                    information about a track, as returned by the Spotify API.

    Returns:
    pandas.DataFrame: A DataFrame containing the processed track information.
    """
    df_dict =
    df_dict = {
        'id':[],
        'date_added':[],
        'name':[],
        'popularity':[],
        'is_local':[],
        'is_explicit':[],
        'duration_ms':[],
        'album_id':[],
        'album_name':[],
        'album_release_date':[],
        'album_release_date_precision':[],
        'artist_id':[],
        'artist_name':[]
    }

    for item in library:
        df_dict['id'].append(item['track']['id']),
        df_dict['date_added'].append(item['added_at']),
        df_dict['name'].append(item['track']['name']),
        df_dict['popularity'].append(item['track']['popularity']),
        df_dict['is_local'].append(item['track']['is_local']),
        df_dict['is_explicit'].append(item['track']['explicit']),
        df_dict['duration_ms'].append(item['track']['duration_ms']),
        df_dict['album_id'].append(item['track']['album']['id']),
        df_dict['album_name'].append(item['track']['album']['name']),
        df_dict['album_release_date'].append(item['track']['album']['release_date']),
        df_dict['album_release_date_precision'].append(item['track']['album']['release_date_precision']),
        df_dict['artist_id'].append(item['track']['artists'][0]['id']),
        df_dict['artist_name'].append(item['track']['artists'][0]['name'])

    return pd.DataFrame(df_dict)

In [38]:
df1 = process_library_tracks(library)
print(df1.shape)
df1.head()

(5742, 14)


,id,date_added,name,popularity,is_local,is_explicit,duration_ms,album_id,album_name,album_release_date,album_release_date_precision,artist_id,artist_name,all_artist_ids
0,5yaCquc7koPqtgj7v0lwHX,2024-05-02T20:12:43Z,Waiting for Tonight,65,False,False,246426,3Gby5NNeNYkMgAnrtEA3lc,On The 6,1999-06-01,day,2DlGxzQSjYe5N6G9nkYghR,Jennifer Lopez,[2DlGxzQSjYe5N6G9nkYghR]
1,550YpmfhHjNO7xqvScSC0i,2024-05-02T14:26:45Z,Take Your Clothes Off When You Dance,38,False,False,92786,3u1f0ziKFCkqcHUQ0AldCU,We're Only In It For The Money,1968-05-04,day,3P2gYnypDVi90ZavnaAhfL,The Mothers Of Invention,[3P2gYnypDVi90ZavnaAhfL]
2,3FiPLOhNiB2Ox4ut1jJxE2,2024-05-01T11:36:35Z,Set The Roof,44,False,False,268779,5DUPSEYAsHkWIF2DZ3rXty,Set The Roof,2023-05-24,day,6olWbKW2VLhFCHfOi0iEDb,Hudson Mohawke,"[6olWbKW2VLhFCHfOi0iEDb, 27JCep1zDO3K8GY50trDo..."
3,4zI0wuANmj9DNkBOgXU5Wh,2024-04-30T08:36:01Z,Two Months Off,46,False,False,548560,5n192ghquuwEFORZEtNyLg,A Hundred Days Off,2002,year,1PXHzxRDiLnjqNrRn2Xbsa,Underworld,[1PXHzxRDiLnjqNrRn2Xbsa]
4,7COnJ7dYdPoxvgJMmAh9OI,2024-04-28T21:31:57Z,Yay Yay Ya Nassini,42,False,False,139598,4OMiPny1FpMBFXmjxsAEDL,The Best Of Georgette Sayegh,1974-01-01,day,1GwinA1uq3G9r8PNK9MoRy,Georgette Sayegh,[1GwinA1uq3G9r8PNK9MoRy]


### Get Audio Features

In [41]:
track_ids = "3CqaTHbiU2nBy3Ar9RnDHN,1dN55SR4aP4EpGdYcQpPb3"
response_json = spotify_api.get_audio_features_several_tracks(track_ids)

In [42]:
response_json

[{'danceability': 0.652,
  'energy': 0.877,
  'key': 6,
  'loudness': -6.281,
  'mode': 0,
  'speechiness': 0.0339,
  'acousticness': 0.00307,
  'instrumentalness': 0.699,
  'liveness': 0.186,
  'valence': 0.562,
  'tempo': 124.024,
  'type': 'audio_features',
  'id': '3CqaTHbiU2nBy3Ar9RnDHN',
  'uri': 'spotify:track:3CqaTHbiU2nBy3Ar9RnDHN',
  'track_href': 'https://api.spotify.com/v1/tracks/3CqaTHbiU2nBy3Ar9RnDHN',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3CqaTHbiU2nBy3Ar9RnDHN',
  'duration_ms': 222316,
  'time_signature': 4},
 {'danceability': 0.736,
  'energy': 0.603,
  'key': 9,
  'loudness': -7.763,
  'mode': 0,
  'speechiness': 0.0276,
  'acousticness': 0.79,
  'instrumentalness': 0.509,
  'liveness': 0.141,
  'valence': 0.187,
  'tempo': 123.957,
  'type': 'audio_features',
  'id': '1dN55SR4aP4EpGdYcQpPb3',
  'uri': 'spotify:track:1dN55SR4aP4EpGdYcQpPb3',
  'track_href': 'https://api.spotify.com/v1/tracks/1dN55SR4aP4EpGdYcQpPb3',
  'analysis_url': 'https://a

In [43]:
def get_library_audio_features(df):
    """
    Retrieves audio features for tracks in a library DataFrame 
    from the Spotify API.

    This function iterates through the rows of the provided DataFrame,
    fetching audio features for tracks in batches of 100 to comply 
    with Spotify API limitations. It then compiles all
    the fetched audio features into a list of dictionaries.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing track information,
                           typically obtained from the 
                           `process_library_tracks` function.

    Returns:
    list: A list of dictionaries containing the audio features
          for each track. If the request fails,
          an error message is returned instead.
    """
    dict_list = []
    
    for n in range(0, (math.ceil(len(df) / 100.0) * 100), 100): # loop through library df rows, 100 at a time
        track_ids = (",").join(df[n:n+100]['id'].to_list()) # get the track ids into a comma-separated string
        dict_list += spotify_api.get_audio_features_several_tracks(track_ids)
        
    return dict_list

In [44]:
audio_features_dicts = get_library_audio_features(df1)

In [49]:
audio_features_dicts[0]

{'danceability': 0.695,
 'energy': 0.945,
 'key': 10,
 'loudness': -6.165,
 'mode': 0,
 'speechiness': 0.0418,
 'acousticness': 0.00341,
 'instrumentalness': 0.0727,
 'liveness': 0.103,
 'valence': 0.84,
 'tempo': 125.013,
 'type': 'audio_features',
 'id': '5yaCquc7koPqtgj7v0lwHX',
 'uri': 'spotify:track:5yaCquc7koPqtgj7v0lwHX',
 'track_href': 'https://api.spotify.com/v1/tracks/5yaCquc7koPqtgj7v0lwHX',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5yaCquc7koPqtgj7v0lwHX',
 'duration_ms': 246427,
 'time_signature': 4}

In [50]:
audio_features = pd.DataFrame(audio_features_dicts)
print(audio_features.shape)
audio_features.head()

(5742, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.695,0.945,10,-6.165,0,0.0418,0.00341,0.072700,0.1030,0.840,125.013,audio_features,5yaCquc7koPqtgj7v0lwHX,spotify:track:5yaCquc7koPqtgj7v0lwHX,https://api.spotify.com/v1/tracks/5yaCquc7koPq...,https://api.spotify.com/v1/audio-analysis/5yaC...,246427,4
1,0.843,0.577,11,-10.433,0,0.0511,0.80600,0.000002,0.1640,0.951,120.688,audio_features,550YpmfhHjNO7xqvScSC0i,spotify:track:550YpmfhHjNO7xqvScSC0i,https://api.spotify.com/v1/tracks/550YpmfhHjNO...,https://api.spotify.com/v1/audio-analysis/550Y...,92787,4
2,0.847,0.677,8,-6.202,1,0.3100,0.00377,0.018800,0.1920,0.550,133.876,audio_features,3FiPLOhNiB2Ox4ut1jJxE2,spotify:track:3FiPLOhNiB2Ox4ut1jJxE2,https://api.spotify.com/v1/tracks/3FiPLOhNiB2O...,https://api.spotify.com/v1/audio-analysis/3FiP...,268779,4
3,0.582,0.896,10,-9.152,0,0.0638,0.00211,0.728000,0.0218,0.665,134.994,audio_features,4zI0wuANmj9DNkBOgXU5Wh,spotify:track:4zI0wuANmj9DNkBOgXU5Wh,https://api.spotify.com/v1/tracks/4zI0wuANmj9D...,https://api.spotify.com/v1/audio-analysis/4zI0...,548560,4
4,0.664,0.506,2,-9.177,0,0.0342,0.58800,0.000000,0.1550,0.750,135.038,audio_features,7COnJ7dYdPoxvgJMmAh9OI,spotify:track:7COnJ7dYdPoxvgJMmAh9OI,https://api.spotify.com/v1/tracks/7COnJ7dYdPox...,https://api.spotify.com/v1/audio-analysis/7COn...,139598,4


In [51]:
audio_features.to_csv(os.path.abspath('..') + "/data/" + "audio_features.csv", index=False)

In [52]:
right = audio_features[[
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'id'
]]

df2 = pd.merge(df1, right, on='id')
print(df2.shape)
df2.head()

(5742, 25)


,id,date_added,name,popularity,is_local,is_explicit,duration_ms,album_id,album_name,album_release_date,album_release_date_precision,artist_id,artist_name,all_artist_ids,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,5yaCquc7koPqtgj7v0lwHX,2024-05-02T20:12:43Z,Waiting for Tonight,65,False,False,246426,3Gby5NNeNYkMgAnrtEA3lc,On The 6,1999-06-01,day,2DlGxzQSjYe5N6G9nkYghR,Jennifer Lopez,[2DlGxzQSjYe5N6G9nkYghR],0.695,0.945,10,-6.165,0,0.0418,0.00341,0.072700,0.1030,0.840,125.013
1,550YpmfhHjNO7xqvScSC0i,2024-05-02T14:26:45Z,Take Your Clothes Off When You Dance,38,False,False,92786,3u1f0ziKFCkqcHUQ0AldCU,We're Only In It For The Money,1968-05-04,day,3P2gYnypDVi90ZavnaAhfL,The Mothers Of Invention,[3P2gYnypDVi90ZavnaAhfL],0.843,0.577,11,-10.433,0,0.0511,0.80600,0.000002,0.1640,0.951,120.688
2,3FiPLOhNiB2Ox4ut1jJxE2,2024-05-01T11:36:35Z,Set The Roof,44,False,False,268779,5DUPSEYAsHkWIF2DZ3rXty,Set The Roof,2023-05-24,day,6olWbKW2VLhFCHfOi0iEDb,Hudson Mohawke,"[6olWbKW2VLhFCHfOi0iEDb, 27JCep1zDO3K8GY50trDo...",0.847,0.677,8,-6.202,1,0.3100,0.00377,0.018800,0.1920,0.550,133.876
3,4zI0wuANmj9DNkBOgXU5Wh,2024-04-30T08:36:01Z,Two Months Off,46,False,False,548560,5n192ghquuwEFORZEtNyLg,A Hundred Days Off,2002,year,1PXHzxRDiLnjqNrRn2Xbsa,Underworld,[1PXHzxRDiLnjqNrRn2Xbsa],0.582,0.896,10,-9.152,0,0.0638,0.00211,0.728000,0.0218,0.665,134.994
4,7COnJ7dYdPoxvgJMmAh9OI,2024-04-28T21:31:57Z,Yay Yay Ya Nassini,42,False,False,139598,4OMiPny1FpMBFXmjxsAEDL,The Best Of Georgette Sayegh,1974-01-01,day,1GwinA1uq3G9r8PNK9MoRy,Georgette Sayegh,[1GwinA1uq3G9r8PNK9MoRy],0.664,0.506,2,-9.177,0,0.0342,0.58800,0.000000,0.1550,0.750,135.038


### Get Album Info

In [56]:
response_json = spotify_api.get_several_albums("2fDJNwprKYSK4ovmRzcoe4,7GK7EmnHe9MdYTDSKHqMg7")

In [57]:
response_json[0].keys()

dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])

In [58]:
response_json[1]['popularity']

33

In [59]:
def get_library_albums(df):
    df_dict = {
        'album_id':[],
        'album_popularity':[],
    }
    album_id_list = df.album_id.unique()
    for n in range(0, (math.ceil(len(album_id_list) / 20) * 20), 20):
        album_ids = (",").join(album_id_list[n:n+20])
        albums = spotify_api.get_several_albums(album_ids)
        for album in albums:
            df_dict['album_id'].append(album['id'])
            df_dict['album_popularity'].append(album['popularity'])
    return  pd.DataFrame(df_dict)

In [60]:
albums = get_library_albums(df2)
print(albums.shape)
albums.head()

(3780, 2)


,album_id,album_popularity
0,3Gby5NNeNYkMgAnrtEA3lc,61
1,3u1f0ziKFCkqcHUQ0AldCU,37
2,5DUPSEYAsHkWIF2DZ3rXty,31
3,5n192ghquuwEFORZEtNyLg,42
4,4OMiPny1FpMBFXmjxsAEDL,33


In [61]:
albums.to_csv("albums.csv", index=False)

In [62]:
df3 = pd.merge(df2, albums, on='album_id')
print(df3.shape)
df3.head()

(5742, 26)


,id,date_added,name,popularity,is_local,is_explicit,duration_ms,album_id,album_name,album_release_date,album_release_date_precision,artist_id,artist_name,all_artist_ids,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,album_popularity
0,5yaCquc7koPqtgj7v0lwHX,2024-05-02T20:12:43Z,Waiting for Tonight,65,False,False,246426,3Gby5NNeNYkMgAnrtEA3lc,On The 6,1999-06-01,day,2DlGxzQSjYe5N6G9nkYghR,Jennifer Lopez,[2DlGxzQSjYe5N6G9nkYghR],0.695,0.945,10,-6.165,0,0.0418,0.00341,0.072700,0.1030,0.840,125.013,61
1,550YpmfhHjNO7xqvScSC0i,2024-05-02T14:26:45Z,Take Your Clothes Off When You Dance,38,False,False,92786,3u1f0ziKFCkqcHUQ0AldCU,We're Only In It For The Money,1968-05-04,day,3P2gYnypDVi90ZavnaAhfL,The Mothers Of Invention,[3P2gYnypDVi90ZavnaAhfL],0.843,0.577,11,-10.433,0,0.0511,0.80600,0.000002,0.1640,0.951,120.688,37
2,3FiPLOhNiB2Ox4ut1jJxE2,2024-05-01T11:36:35Z,Set The Roof,44,False,False,268779,5DUPSEYAsHkWIF2DZ3rXty,Set The Roof,2023-05-24,day,6olWbKW2VLhFCHfOi0iEDb,Hudson Mohawke,"[6olWbKW2VLhFCHfOi0iEDb, 27JCep1zDO3K8GY50trDo...",0.847,0.677,8,-6.202,1,0.3100,0.00377,0.018800,0.1920,0.550,133.876,31
3,4zI0wuANmj9DNkBOgXU5Wh,2024-04-30T08:36:01Z,Two Months Off,46,False,False,548560,5n192ghquuwEFORZEtNyLg,A Hundred Days Off,2002,year,1PXHzxRDiLnjqNrRn2Xbsa,Underworld,[1PXHzxRDiLnjqNrRn2Xbsa],0.582,0.896,10,-9.152,0,0.0638,0.00211,0.728000,0.0218,0.665,134.994,42
4,7COnJ7dYdPoxvgJMmAh9OI,2024-04-28T21:31:57Z,Yay Yay Ya Nassini,42,False,False,139598,4OMiPny1FpMBFXmjxsAEDL,The Best Of Georgette Sayegh,1974-01-01,day,1GwinA1uq3G9r8PNK9MoRy,Georgette Sayegh,[1GwinA1uq3G9r8PNK9MoRy],0.664,0.506,2,-9.177,0,0.0342,0.58800,0.000000,0.1550,0.750,135.038,33


### Get Artist Info

In [65]:
response_json = spotify_api.get_several_artists("2MPHBxznH1fj59jbOWY38u,7A0awCXkE1FtSU8B0qwOJQ")

In [66]:
response_json[0].keys()

dict_keys(['external_urls', 'followers', 'genres', 'href', 'id', 'images', 'name', 'popularity', 'type', 'uri'])

In [67]:
response_json[1]['genres']

['electronica', 'future garage', 'indie soul', 'indietronica']

In [68]:
def get_library_artists(df):
    df_dict = {
        'artist_id':[],
        'artist_popularity':[],
        'artist_genres':[],
    }
    artist_id_list = df.artist_id.unique()
    for n in range(0, (math.ceil(len(artist_id_list) / 50) * 50), 50):
        artist_ids = (",").join(artist_id_list[n:n+50])
        artists = spotify_api.get_several_artists(artist_ids)
        for artist in artists:
            df_dict['artist_id'].append(artist['id'])
            df_dict['artist_popularity'].append(artist['popularity'])
            df_dict['artist_genres'].append(artist['genres'])
    return  pd.DataFrame(df_dict)

In [69]:
artists = get_library_artists(df3)
print(artists.shape)
artists.head()

(2185, 3)


,artist_id,artist_popularity,artist_genres
0,2DlGxzQSjYe5N6G9nkYghR,72,"[dance pop, pop, urban contemporary]"
1,3P2gYnypDVi90ZavnaAhfL,41,"[art rock, canterbury scene, experimental, exp..."
2,6olWbKW2VLhFCHfOi0iEDb,42,"[instrumental hip hop, purple sound, scottish ..."
3,1PXHzxRDiLnjqNrRn2Xbsa,53,"[big beat, electronica]"
4,1GwinA1uq3G9r8PNK9MoRy,26,[]


In [70]:
artists.to_csv(os.path.abspath('..') + "/data/" + "artists.csv", index=False)

In [71]:
df4 = pd.merge(df3, artists, on='artist_id')
print(df4.shape)
df4.head()

(5742, 28)


,id,date_added,name,popularity,is_local,is_explicit,duration_ms,album_id,album_name,album_release_date,album_release_date_precision,artist_id,artist_name,all_artist_ids,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,album_popularity,artist_popularity,artist_genres
0,5yaCquc7koPqtgj7v0lwHX,2024-05-02T20:12:43Z,Waiting for Tonight,65,False,False,246426,3Gby5NNeNYkMgAnrtEA3lc,On The 6,1999-06-01,day,2DlGxzQSjYe5N6G9nkYghR,Jennifer Lopez,[2DlGxzQSjYe5N6G9nkYghR],0.695,0.945,10,-6.165,0,0.0418,0.00341,0.072700,0.1030,0.840,125.013,61,72,"[dance pop, pop, urban contemporary]"
1,550YpmfhHjNO7xqvScSC0i,2024-05-02T14:26:45Z,Take Your Clothes Off When You Dance,38,False,False,92786,3u1f0ziKFCkqcHUQ0AldCU,We're Only In It For The Money,1968-05-04,day,3P2gYnypDVi90ZavnaAhfL,The Mothers Of Invention,[3P2gYnypDVi90ZavnaAhfL],0.843,0.577,11,-10.433,0,0.0511,0.80600,0.000002,0.1640,0.951,120.688,37,41,"[art rock, canterbury scene, experimental, exp..."
2,3FiPLOhNiB2Ox4ut1jJxE2,2024-05-01T11:36:35Z,Set The Roof,44,False,False,268779,5DUPSEYAsHkWIF2DZ3rXty,Set The Roof,2023-05-24,day,6olWbKW2VLhFCHfOi0iEDb,Hudson Mohawke,"[6olWbKW2VLhFCHfOi0iEDb, 27JCep1zDO3K8GY50trDo...",0.847,0.677,8,-6.202,1,0.3100,0.00377,0.018800,0.1920,0.550,133.876,31,42,"[instrumental hip hop, purple sound, scottish ..."
3,4zI0wuANmj9DNkBOgXU5Wh,2024-04-30T08:36:01Z,Two Months Off,46,False,False,548560,5n192ghquuwEFORZEtNyLg,A Hundred Days Off,2002,year,1PXHzxRDiLnjqNrRn2Xbsa,Underworld,[1PXHzxRDiLnjqNrRn2Xbsa],0.582,0.896,10,-9.152,0,0.0638,0.00211,0.728000,0.0218,0.665,134.994,42,53,"[big beat, electronica]"
4,7COnJ7dYdPoxvgJMmAh9OI,2024-04-28T21:31:57Z,Yay Yay Ya Nassini,42,False,False,139598,4OMiPny1FpMBFXmjxsAEDL,The Best Of Georgette Sayegh,1974-01-01,day,1GwinA1uq3G9r8PNK9MoRy,Georgette Sayegh,[1GwinA1uq3G9r8PNK9MoRy],0.664,0.506,2,-9.177,0,0.0342,0.58800,0.000000,0.1550,0.750,135.038,33,26,[]


In [72]:
df4.to_csv(os.path.abspath('..') + "/data/" + "liked_songs_raw.csv", index=False)

## 4. Cleaning the Data

In [88]:
df = pd.read_csv(os.path.abspath('..') + "/data/" + "liked_songs_raw.csv")
df.head()

,id,date_added,name,popularity,is_local,is_explicit,duration_ms,album_id,album_name,album_release_date,album_release_date_precision,artist_id,artist_name,all_artist_ids,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,album_popularity,artist_popularity,artist_genres
0,5yaCquc7koPqtgj7v0lwHX,2024-05-02T20:12:43Z,Waiting for Tonight,65,False,False,246426,3Gby5NNeNYkMgAnrtEA3lc,On The 6,1999-06-01,day,2DlGxzQSjYe5N6G9nkYghR,Jennifer Lopez,[2DlGxzQSjYe5N6G9nkYghR],0.695,0.945,10,-6.165,0,0.0418,0.00341,0.072700,0.1030,0.840,125.013,61,72,"[dance pop, pop, urban contemporary]"
1,550YpmfhHjNO7xqvScSC0i,2024-05-02T14:26:45Z,Take Your Clothes Off When You Dance,38,False,False,92786,3u1f0ziKFCkqcHUQ0AldCU,We're Only In It For The Money,1968-05-04,day,3P2gYnypDVi90ZavnaAhfL,The Mothers Of Invention,[3P2gYnypDVi90ZavnaAhfL],0.843,0.577,11,-10.433,0,0.0511,0.80600,0.000002,0.1640,0.951,120.688,37,41,"[art rock, canterbury scene, experimental, exp..."
2,3FiPLOhNiB2Ox4ut1jJxE2,2024-05-01T11:36:35Z,Set The Roof,44,False,False,268779,5DUPSEYAsHkWIF2DZ3rXty,Set The Roof,2023-05-24,day,6olWbKW2VLhFCHfOi0iEDb,Hudson Mohawke,"[6olWbKW2VLhFCHfOi0iEDb, 27JCep1zDO3K8GY50trDo...",0.847,0.677,8,-6.202,1,0.3100,0.00377,0.018800,0.1920,0.550,133.876,31,42,"[instrumental hip hop, purple sound, scottish ..."
3,4zI0wuANmj9DNkBOgXU5Wh,2024-04-30T08:36:01Z,Two Months Off,46,False,False,548560,5n192ghquuwEFORZEtNyLg,A Hundred Days Off,2002,year,1PXHzxRDiLnjqNrRn2Xbsa,Underworld,[1PXHzxRDiLnjqNrRn2Xbsa],0.582,0.896,10,-9.152,0,0.0638,0.00211,0.728000,0.0218,0.665,134.994,42,53,"[big beat, electronica]"
4,7COnJ7dYdPoxvgJMmAh9OI,2024-04-28T21:31:57Z,Yay Yay Ya Nassini,42,False,False,139598,4OMiPny1FpMBFXmjxsAEDL,The Best Of Georgette Sayegh,1974-01-01,day,1GwinA1uq3G9r8PNK9MoRy,Georgette Sayegh,[1GwinA1uq3G9r8PNK9MoRy],0.664,0.506,2,-9.177,0,0.0342,0.58800,0.000000,0.1550,0.750,135.038,33,26,[]


In [102]:
df.date_added = pd.to_datetime(df.date_added)
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month
df['time_added'] = df['date_added'].dt.time

df['day_of_week_added'] = df['date_added'].dt.dayofweek

days_of_week_mapping = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}

df['day_of_week_added'] = df['day_of_week_added'].apply(lambda x: days_of_week_mapping[x])

    month_mapping = {
        1: 'January',
        2: 'February',
        3: 'March',
        4: 'April',
        5: 'May',
        6: 'June',
        7: 'July',
        8: 'August',
        9: 'September',
        10: 'October',
        11: 'November',
        12: 'December'
    }
    
    
    df['month_added'] = df['month_added'].apply(lambda x: month_mapping[x])

df['date_added'] = df['date_added'].dt.date

In [91]:
df['duration_ms'] = df['duration_ms'] / 1000
df = df.rename(columns={'duration_ms':'duration_s'})
df['duration_min'] = df['duration_s'] / 60

In [92]:
df = df[['id',
 'name',
 'popularity',
 'is_local',
 'is_explicit',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'date_added',
 'year_added',
 'month_added',
 'day_of_week_added',
 'time_added',
 'duration_s',
 'duration_min',
 'album_id',
 'album_name',
 'album_popularity',
 'album_release_date',
 'album_release_date_precision',
 'artist_id',
 'artist_name',
 'artist_popularity',
 'artist_genres']]
print(df.shape)
df.head()

(5742, 33)


,id,name,popularity,is_local,is_explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,date_added,year_added,month_added,day_of_week_added,time_added,duration_s,duration_min,album_id,album_name,album_popularity,album_release_date,album_release_date_precision,artist_id,artist_name,artist_popularity,all_artist_ids,artist_genres
0,5yaCquc7koPqtgj7v0lwHX,Waiting for Tonight,65,False,False,0.695,0.945,10,-6.165,0,0.0418,0.00341,0.072700,0.1030,0.840,125.013,2024-05-02 20:12:43+00:00,2024,5,Thursday,20:12:43,246.426,4.107100,3Gby5NNeNYkMgAnrtEA3lc,On The 6,61,1999-06-01,day,2DlGxzQSjYe5N6G9nkYghR,Jennifer Lopez,72,[2DlGxzQSjYe5N6G9nkYghR],"[dance pop, pop, urban contemporary]"
1,550YpmfhHjNO7xqvScSC0i,Take Your Clothes Off When You Dance,38,False,False,0.843,0.577,11,-10.433,0,0.0511,0.80600,0.000002,0.1640,0.951,120.688,2024-05-02 14:26:45+00:00,2024,5,Thursday,14:26:45,92.786,1.546433,3u1f0ziKFCkqcHUQ0AldCU,We're Only In It For The Money,37,1968-05-04,day,3P2gYnypDVi90ZavnaAhfL,The Mothers Of Invention,41,[3P2gYnypDVi90ZavnaAhfL],"[art rock, canterbury scene, experimental, exp..."
2,3FiPLOhNiB2Ox4ut1jJxE2,Set The Roof,44,False,False,0.847,0.677,8,-6.202,1,0.3100,0.00377,0.018800,0.1920,0.550,133.876,2024-05-01 11:36:35+00:00,2024,5,Wednesday,11:36:35,268.779,4.479650,5DUPSEYAsHkWIF2DZ3rXty,Set The Roof,31,2023-05-24,day,6olWbKW2VLhFCHfOi0iEDb,Hudson Mohawke,42,"[6olWbKW2VLhFCHfOi0iEDb, 27JCep1zDO3K8GY50trDo...","[instrumental hip hop, purple sound, scottish ..."
3,4zI0wuANmj9DNkBOgXU5Wh,Two Months Off,46,False,False,0.582,0.896,10,-9.152,0,0.0638,0.00211,0.728000,0.0218,0.665,134.994,2024-04-30 08:36:01+00:00,2024,4,Tuesday,08:36:01,548.560,9.142667,5n192ghquuwEFORZEtNyLg,A Hundred Days Off,42,2002,year,1PXHzxRDiLnjqNrRn2Xbsa,Underworld,53,[1PXHzxRDiLnjqNrRn2Xbsa],"[big beat, electronica]"
4,7COnJ7dYdPoxvgJMmAh9OI,Yay Yay Ya Nassini,42,False,False,0.664,0.506,2,-9.177,0,0.0342,0.58800,0.000000,0.1550,0.750,135.038,2024-04-28 21:31:57+00:00,2024,4,Sunday,21:31:57,139.598,2.326633,4OMiPny1FpMBFXmjxsAEDL,The Best Of Georgette Sayegh,33,1974-01-01,day,1GwinA1uq3G9r8PNK9MoRy,Georgette Sayegh,26,[1GwinA1uq3G9r8PNK9MoRy],[]


### Genres

In [94]:
lists_of_genres = df['artist_genres'].to_list()
genres = sorted(set(list(chain.from_iterable(lists_of_genres))))

In [95]:
genres[:5]

['2-step', 'abstract', 'abstract beats', 'abstract hip hop', 'acid house']

In [96]:
genres_df = df[['artist_id', 'artist_genres']].copy()
genres_df.head()

,artist_id,artist_genres
0,2DlGxzQSjYe5N6G9nkYghR,"[dance pop, pop, urban contemporary]"
1,3P2gYnypDVi90ZavnaAhfL,"[art rock, canterbury scene, experimental, exp..."
2,6olWbKW2VLhFCHfOi0iEDb,"[instrumental hip hop, purple sound, scottish ..."
3,1PXHzxRDiLnjqNrRn2Xbsa,"[big beat, electronica]"
4,1GwinA1uq3G9r8PNK9MoRy,[]


In [97]:
for genre in genres:
    genres_df[genre] = genres_df['artist_genres'].apply(lambda x: 1 if genre in x else 0)
genres_df.drop(columns='artist_genres', inplace=True)
genres_df.drop_duplicates(inplace=True)
print(genres_df.shape)
genres_df.head()

(2185, 936)


,artist_id,2-step,abstract,abstract beats,abstract hip hop,acid house,acid jazz,acid rock,acid techno,acousmatic,acoustic blues,adult standards,african electronic,african percussion,afro house,afro psych,afro r&b,afro soul,afro-cuban percussion,afro-funk,afrobeat,afrobeat brasileiro,afrobeat fusion,afrobeats,afrofuturism,afrofuturismo brasileiro,afropop,afroswing,alabama indie,alabama rap,album rock,alte,alternative americana,alternative dance,alternative hip hop,alternative metal,alternative pop rock,alternative r&b,alternative rock,ambient,ambient country,ambient house,ambient idm,ambient pop,ambient techno,american contemporary classical,arab alternative,arab groove,arabic hip hop,argentine rock,arkansas country,armenian hip hop,art pop,art punk,art rock,atl hip hop,atl trap,auckland indie,aussie drill,aussietronica,australian alternative rock,australian ambient,australian dance,australian electropop,australian experimental,australian hip hop,australian house,australian indie,australian indie rock,australian pop,australian psych,australian r&b,australian reggae fusion,australian rock,australian surf rock,australian trap,austrian dnb,avant-garde jazz,azonto,azontobeats,balearic,barbadian pop,bardcore,baroque pop,bass house,bass trap,bass trip,bath indie,bay area indie,bboy,beach music,beatlesque,bebop,bedroom pop,bedroom soul,belgian edm,belgian hip hop,belgian pop,belly dance,benga,berlin school,big beat,birmingham grime,birmingham hip hop,birmingham indie,birmingham metal,black comedy,black punk,bleep techno,blues,blues rock,bmore,bolero,boogaloo,boogie,boom bap,boom bap espanol,bossa nova,bossa nova jazz,boston hip hop,boston rock,boy band,braindance,brass band,brass band pop,brazilian boogie,brazilian jazz,brazilian soul,breakbeat,breaks,brill building pop,bristol electronic,brit funk,british alternative rock,british blues,british comedy,british dance band,british experimental,british invasion,british jazz,british math rock,british singer-songwriter,british soul,britpop,broken beat,bronx hip hop,brooklyn drill,brooklyn indie,brostep,bubble trance,bubblegum pop,buffalo hip hop,bulgarian electronic,cali rap,calypso,cambodian rock,canadian americana,canadian comedy,canadian contemporary country,canadian contemporary r&b,canadian country,canadian electronic,canadian electropop,canadian hip hop,canadian house,canadian indigenous hip hop,canadian indigenous music,canadian latin,canadian modern jazz,canadian pop,canadian singer-songwriter,canadian trap,candy pop,canterbury scene,canzone d'autore,cascadia psych,chamber pop,chamber psych,channel islands indie,chanson,chicago blues,chicago bop,chicago drill,chicago house,chicago rap,chicago soul,chicha,chill abstract hip hop,chill baile,chill drill,chill r&b,chillhop,chillwave,chimurenga,cincinnati rap,city pop,classic afrobeat,classic bollywood,classic canadian rock,classic city pop,classic country pop,classic dubstep,classic garage rock,classic girl group,classic house,classic italian pop,classic japanese jazz,classic polish pop,classic portuguese pop,classic progressive house,classic rock,classic soul,classic turkish pop,classic uk pop,classical,cloud rap,collage pop,colombian pop,comedy,complextro,compositional ambient,conscious hip hop,contemporary classical,contemporary country,contemporary jazz,contemporary post-bop,contemporary r&b,cool jazz,cosmic american,country,country dawn,country rock,crank wave,cumbia funk,dance pop,dance rock,dance-punk,dancefloor dnb,dancehall,dancehall queen,danish electronic,danish indie pop,danish modern jazz,danish techno,dark disco,dark plugg,dark techno,dark trap,deconstructed club,deep disco,deep disco house,deep euro house,deep funk,deep funk house,deep house,deep idm,deep liquid bass,deep minimal techno,deep soul house,deep tech house,deep techno,deep underground hip hop,delta blues,denton tx indie,desert blues,destroy techno,detroit hip hop,detroit house,detroit rock,detroit techno,dfw rap,dirty south rap,disco,disco house,diva house,dmv

In [98]:
df_clean = pd.merge(df, genres_df, on='artist_id', how='inner')
df_clean.drop(columns='artist_genres', inplace=True)
print(df_clean.shape)
df_clean.head()

(5742, 967)


,id,name,popularity,is_local,is_explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,date_added,year_added,month_added,day_of_week_added,time_added,duration_s,duration_min,album_id,album_name,album_popularity,album_release_date,album_release_date_precision,artist_id,artist_name,artist_popularity,all_artist_ids,2-step,abstract,abstract beats,abstract hip hop,acid house,acid jazz,acid rock,acid techno,acousmatic,acoustic blues,adult standards,african electronic,african percussion,afro house,afro psych,afro r&b,afro soul,afro-cuban percussion,afro-funk,afrobeat,afrobeat brasileiro,afrobeat fusion,afrobeats,afrofuturism,afrofuturismo brasileiro,afropop,afroswing,alabama indie,alabama rap,album rock,alte,alternative americana,alternative dance,alternative hip hop,alternative metal,alternative pop rock,alternative r&b,alternative rock,ambient,ambient country,ambient house,ambient idm,ambient pop,ambient techno,american contemporary classical,arab alternative,arab groove,arabic hip hop,argentine rock,arkansas country,armenian hip hop,art pop,art punk,art rock,atl hip hop,atl trap,auckland indie,aussie drill,aussietronica,australian alternative rock,australian ambient,australian dance,australian electropop,australian experimental,australian hip hop,australian house,australian indie,australian indie rock,australian pop,australian psych,australian r&b,australian reggae fusion,australian rock,australian surf rock,australian trap,austrian dnb,avant-garde jazz,azonto,azontobeats,balearic,barbadian pop,bardcore,baroque pop,bass house,bass trap,bass trip,bath indie,bay area indie,bboy,beach music,beatlesque,bebop,bedroom pop,bedroom soul,belgian edm,belgian hip hop,belgian pop,belly dance,benga,berlin school,big beat,birmingham grime,birmingham hip hop,birmingham indie,birmingham metal,black comedy,black punk,bleep techno,blues,blues rock,bmore,bolero,boogaloo,boogie,boom bap,boom bap espanol,bossa nova,bossa nova jazz,boston hip hop,boston rock,boy band,braindance,brass band,brass band pop,brazilian boogie,brazilian jazz,brazilian soul,breakbeat,breaks,brill building pop,bristol electronic,brit funk,british alternative rock,british blues,british comedy,british dance band,british experimental,british invasion,british jazz,british math rock,british singer-songwriter,british soul,britpop,broken beat,bronx hip hop,brooklyn drill,brooklyn indie,brostep,bubble trance,bubblegum pop,buffalo hip hop,bulgarian electronic,cali rap,calypso,cambodian rock,canadian americana,canadian comedy,canadian contemporary country,canadian contemporary r&b,canadian country,canadian electronic,canadian electropop,canadian hip hop,canadian house,canadian indigenous hip hop,canadian indigenous music,canadian latin,canadian modern jazz,canadian pop,canadian singer-songwriter,canadian trap,candy pop,canterbury scene,canzone d'autore,cascadia psych,chamber pop,chamber psych,channel islands indie,chanson,chicago blues,chicago bop,chicago drill,chicago house,chicago rap,chicago soul,chicha,chill abstract hip hop,chill baile,chill drill,chill r&b,chillhop,chillwave,chimurenga,cincinnati rap,city pop,classic afrobeat,classic bollywood,classic canadian rock,classic city pop,classic country pop,classic dubstep,classic garage rock,classic girl group,classic house,classic italian pop,classic japanese jazz,classic polish pop,classic portuguese pop,classic progressive house,classic rock,classic soul,classic turkish pop,classic uk pop,classical,cloud rap,collage pop,colombian pop,comedy,complextro,compositional ambient,conscious hip hop,contemporary classical,contemporary country,contemporary jazz,contemporary post-bop,contemporary r&b,cool jazz,cosmic american,country,country dawn,country rock,crank wave,cumbia funk,dance pop,dance rock,dance-punk,dancefloor dnb,dancehall,dancehall queen,danish electronic,danish indie pop,danish modern jazz,danish techno,dark disco,dark plugg,dark techno,dark trap,deconstructed club,deep d